In [2]:
pip install -U datasets huggingface_hub fsspec

  Obtaining dependency information for fsspec from https://files.pythonhosted.org/packages/bb/61/78c7b3851add1481b048b5fdc29067397a1784e2910592bc81bb3f608635/fsspec-2025.5.1-py3-none-any.whl.metadata
  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import datasets, huggingface_hub, fsspec
print(datasets.__version__, huggingface_hub.__version__, fsspec.__version__)

3.6.0 0.32.5 2025.3.0


In [22]:
from datasets import load_dataset
from transformers import (GPT2Tokenizer, GPT2ForSequenceClassification,
                          TrainingArguments, Trainer)

# 1. Load a labeled tweet dataset
dataset = load_dataset("mteb/tweet_sentiment_extraction")

# 2. Tokenize
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
def preprocess(ex):
    return tokenizer(ex["text"], truncation=True, padding="max_length", max_length=128)
tokenized = dataset.map(preprocess, batched=True)
print(tokenized)
print(tokenized['train'][0])

# Use a small subset for faster iteration
train = tokenized["train"].shuffle(seed=42).select(range(2000))
test = tokenized["test"].shuffle(seed=42).select(range(1000))
print(train.features)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 27481
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 3534
    })
})
{'text': ' I`d have responded, if I were going', 'label': 1, 'input_ids': [314, 63, 67, 423, 7082, 11, 611, 314, 547, 1016, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256

In [5]:
# 3. Load pretrained GPT-2 for classification
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=3)


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   return metric.compute(predictions=predictions, references=labels)

In [7]:
pip install 'accelerate>=0.26.0'


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
   output_dir="test_trainer",
   #evaluation_strategy="epoch",
   per_device_train_batch_size=1,
   per_device_eval_batch_size=1,
   gradient_accumulation_steps=4
   )


trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=train,
   eval_dataset=test,
   compute_metrics=compute_metrics,
)

trainer.train()

Step,Training Loss
500,1.127500
1000,0.775100
1500,0.613200


TrainOutput(global_step=1500, training_loss=0.8386027425130208, metrics={'train_runtime': 1133.217, 'train_samples_per_second': 5.295, 'train_steps_per_second': 1.324, 'total_flos': 391948664832000.0, 'train_loss': 0.8386027425130208, 'epoch': 3.0})

In [10]:
trainer.evaluate()


{'eval_loss': 0.824856162071228,
 'eval_accuracy': 0.753,
 'eval_runtime': 36.1491,
 'eval_samples_per_second': 27.663,
 'eval_steps_per_second': 27.663,
 'epoch': 3.0}

In [21]:
import torch
sample = "I don't like this new phone! but there are things which I really liked"
inputs = tokenizer(sample, return_tensors="pt").to(model.device)
model.eval()
with torch.no_grad():
    logits = model(**inputs).logits
    print(logits.shape)
pred = logits.argmax(dim=-1).item()
label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}
print("Sample prediction:", label_map[pred])

torch.Size([1, 3])
Sample prediction: Negative
